# 近くの場所に関する関数の呼び出し：Google Places APIとカスタマープロファイルを活用する

このノートブックは、Google Places APIとカスタマープロファイルの統合を中心に、位置情報を活用した検索を向上させることを目的としています。私たちのアプローチは、Google Places APIをユーザーの好みと組み合わせて、場所の発見をより個人的で関連性のあるものにすることです。このインスタンスではGoogle Places APIに焦点を当てていますが、同様の方法で探索および適用できる他の多くのAPIが存在することに注意してください。

私たちは以下の3つの主要なコンポーネントの適用を探ります：

- カスタマープロファイル：このモックプロファイルは、場所の種類（例：レストラン、公園、博物館）、予算、好みの評価、その他の具体的な要件に関する個々の好みを捉えます。

- Google Places API：このAPIは、近くの場所に関するリアルタイムのデータを提供します。評価、施設の種類、コストなど、周囲の場所からさまざまなデータポイントを考慮に入れています。

- 関数の呼び出し：「お腹がすいた」といった単一のコマンドや「博物館に行きたい」といったコマンドを発行すると、ユーザープロファイルデータとGoogle Places APIを組み合わせて適切な施設を特定する関数がアクティブ化されます。

このノートブックは、次の2つの主要なユースケースを紹介しています：

- プロファイルベースの推奨事項：ユーザープロファイルを作成し、個々の好みに基づいて場所の推奨事項を行う方法を学びます。

- 関数呼び出しを伴うAPI統合：Google Places APIを効果的に統合して呼び出す方法を理解し、関数呼び出しを使用してさまざまな場所のリアルタイムデータを取得する方法を理解します。

このシステムは非常に柔軟ですが、その効果はユーザーの好みと利用可能な場所データによって異なる可能性があります。このノートブックの目的上、カスタマーデータは架空であり、場所はハードコードされています。

## セットアップ

Google Places API

Google Places APIを使用するには、2つのものが必要です：

- Googleアカウント：すでに持っていない場合は、Googleアカウントを作成する必要があります。

- Google Places APIキー：APIキーは、プロジェクトに関連するリクエストを認証するために使用される一意の識別子であり、使用料金の目的で使用されます。APIキーは[Google Cloud Console](https://console.cloud.google.com/getting-started?authuser=1)から取得できます。


Google Places APIは有料サービスであり、API呼び出しの数に関連する費用が発生します。予期せぬ料金を避けるために使用状況を追跡してください。


また、requestsライブラリも必要です。次のコマンドを使用してダウンロードできます：

```python
pip install requests
```

In [1]:
import json
import openai
import os
import requests

このコードスニペットでは、`fetch_customer_profile` という関数を定義しています。この関数は `user_id` を受け取り、モックのユーザープロファイルを返します。

この関数は、データベースからユーザーデータを取得するAPI呼び出しをシミュレートしています。このデモでは、ハードコードされたデータを使用しています。ユーザープロファイルには、ユーザーの位置情報（この例ではゴールデンゲートブリッジの座標に設定されています）、食べ物やアクティビティの好み、アプリの利用状況メトリクス、最近の対話、ユーザーランクなど、さまざまな詳細が含まれています。

本番環境では、このハードコードされたデータをユーザーデータベースへの実際のAPI呼び出しに置き換える必要があります。


In [2]:
def fetch_customer_profile(user_id):
    # You can replace this with a real API call in the production code
    if user_id == "user1234":
        return {
            "name": "John Doe",
            "location": {
                "latitude": 37.7955,
                "longitude": -122.4026,
            },
            "preferences": {
                "food": ["Italian", "Sushi"],
                "activities": ["Hiking", "Reading"],
            },
            "behavioral_metrics": {
                "app_usage": {
                    "daily": 2,  # hours
                    "weekly": 14  # hours
                },
                "favourite_post_categories": ["Nature", "Food", "Books"],
                "active_time": "Evening",
            },
            "recent_searches": ["Italian restaurants nearby", "Book clubs"],
            "recent_interactions": ["Liked a post about 'Best Pizzas in New York'", "Commented on a post about 'Central Park Trails'"],
            "user_rank": "Gold",  # based on some internal ranking system
        }
    else:
        return None


## Google Places APIからデータをリクエストおよび処理する

関数call_google_places_apiは、Google Places APIから情報をリクエストし、指定された場所の種類（place_type）およびオプションの食事の嗜好（food_preference）に基づいてトップ2の場所のリストを提供するためのものです。これは有料サービスであるため、この関数は使用量を管理するためにトップ2の結果に制限しています。ただし、必要に応じて任意の数の結果を取得するように変更することができます。

この関数は、ハードコーディングされた場所（Transamerica Pyramidの座標に設定されています）、Google APIキー、および特定のリクエストパラメータで構成されています。place_typeに応じて、適切なAPIリクエストURLを組み立てます。place_typeがレストランであり、food_preferenceが指定されている場合、それはAPIリクエストに含まれます。

GETリクエストを送信した後、関数は応答のステータスをチェックします。成功した場合、get_place_details関数を使用して関連する詳細情報を抽出し、人間が読みやすい形式で返します。リクエストが失敗した場合、デバッグ用にエラーを表示します。

get_place_details関数は、place_idが与えられた場所についての詳細情報を取得するために使用されます。Google Place Details APIにGETリクエストを送信し、リクエストが成功した場合は結果を返します。リクエストが失敗した場合、デバッグ用にエラーを表示します。

どちらの関数も例外を処理し、何かがうまくいかない場合はエラーメッセージを返します。

In [3]:
def get_place_details(place_id, api_key):
    URL = f"https://maps.googleapis.com/maps/api/place/details/json?place_id={place_id}&key={api_key}"
    response = requests.get(URL)
    if response.status_code == 200:
        result = json.loads(response.content)["result"]
        return result
    else:
        print(f"Google Place Details API request failed with status code {response.status_code}")
        print(f"Response content: {response.content}")
        return None

In [4]:
def call_google_places_api(user_id, place_type, food_preference=None):
    try:
        # Fetch customer profile
        customer_profile = fetch_customer_profile(user_id)
        if customer_profile is None:
            return "I couldn't find your profile. Could you please verify your user ID?"

        # Get location from customer profile
        lat = customer_profile["location"]["latitude"]
        lng = customer_profile["location"]["longitude"]

        API_KEY = os.getenv('GOOGLE_PLACES_API_KEY')  # retrieve API key from environment variable
        LOCATION = f"{lat},{lng}"
        RADIUS = 500  # search within a radius of 500 meters
        TYPE = place_type

        # If the place_type is restaurant and food_preference is not None, include it in the API request
        if place_type == 'restaurant' and food_preference:
            URL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type={TYPE}&keyword={food_preference}&key={API_KEY}"
        else:
            URL = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={LOCATION}&radius={RADIUS}&type={TYPE}&key={API_KEY}"

        response = requests.get(URL)
        if response.status_code == 200:
            results = json.loads(response.content)["results"]
            places = []
            for place in results[:2]:  # limit to top 2 results
                place_id = place.get("place_id")
                place_details = get_place_details(place_id, API_KEY)  # Get the details of the place
                
                place_name = place_details.get("name", "N/A")
                place_types = next((t for t in place_details.get("types", []) if t not in ["food", "point_of_interest"]), "N/A")  # Get the first type of the place, excluding "food" and "point_of_interest"
                place_rating = place_details.get("rating", "N/A")  # Get the rating of the place
                total_ratings = place_details.get("user_ratings_total", "N/A")  # Get the total number of ratings
                place_address = place_details.get("vicinity", "N/A")  # Get the vicinity of the place
                
                if ',' in place_address:  # If the address contains a comma
                    street_address = place_address.split(',')[0]  # Split by comma and keep only the first part
                else:
                    street_address = place_address
                
                # Prepare the output string for this place
                place_info = f"{place_name} is a {place_types} located at {street_address}. It has a rating of {place_rating} based on {total_ratings} user reviews."
                
                places.append(place_info)

            return places
        else:
            print(f"Google Places API request failed with status code {response.status_code}")
            print(f"Response content: {response.content}")  # print out the response content for debugging
            return []
    except Exception as e:
        print(f"Error during the Google Places API call: {e}")
        return []

## GPT-3.5-TurboとGoogle Places APIを使ってユーザー特有のおすすめ情報を生成

`provide_user_specific_recommendations` 関数は、GPT-3.5-TurboとGoogle Places APIと連携して、ユーザーの嗜好と位置に合わせたレスポンスを提供します。

まず、`user_id` を使用して顧客のプロフィールを取得します。プロフィールが見つからない場合は、エラーメッセージを返します。

有効なプロフィールがある場合、顧客の食べ物の嗜好を抽出し、OpenAIモデルと対話します。初期のシステムメッセージを提供して、AIモデルにその役割、ユーザーの嗜好、およびGoogle Places API関数の使用法についての文脈を与えます。

ユーザーからの入力もモデルにメッセージとして送信され、必要に応じてAIモデルが呼び出す`call_google_places_api` 関数が`functions` パラメータで定義されます。

最後に、モデルのレスポンスを処理します。モデルがGoogle Places APIに関数呼び出しを行う場合、適切な引数で関数が実行され、近くの場所の名前が返されます。そのような場所がないか、リクエストが理解されない場合は、適切なエラーメッセージが返されます。

In [14]:
def provide_user_specific_recommendations(user_input, user_id):
    customer_profile = fetch_customer_profile(user_id)
    if customer_profile is None:
        return "I couldn't find your profile. Could you please verify your user ID?"
    
    customer_profile_str = json.dumps(customer_profile)

    food_preference = customer_profile.get('preferences', {}).get('food', [])[0] if customer_profile.get('preferences', {}).get('food') else None
  

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
    {
        "role": "system",
        "content": f"You are a sophisticated AI assistant, a specialist in user intent detection and interpretation. Your task is to perceive and respond to the user's needs, even when they're expressed in an indirect or direct manner. You excel in recognizing subtle cues: for example, if a user states they are 'hungry', you should assume they are seeking nearby dining options such as a restaurant or a cafe. If they indicate feeling 'tired', 'weary', or mention a long journey, interpret this as a request for accommodation options like hotels or guest houses. However, remember to navigate the fine line of interpretation and assumption: if a user's intent is unclear or can be interpreted in multiple ways, do not hesitate to politely ask for additional clarification. Make sure to tailor your responses to the user based on their preferences and past experiences which can be found here {customer_profile_str}"
    },
    {"role": "user", "content": user_input}
],
        temperature=0,
        functions=[
            {
                "name": "call_google_places_api",
                "description": "This function calls the Google Places API to find the top places of a specified type near a specific location. It can be used when a user expresses a need (e.g., feeling hungry or tired) or wants to find a certain type of place (e.g., restaurant or hotel).",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "place_type": {
                            "type": "string",
                            "description": "The type of place to search for."
                        }
                    }
                },
                "result": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                }
            }
        ],
    )

    if 'message' in response.choices[0] and 'function_call' in response.choices[0]['message']:
        function_call = response.choices[0]['message']['function_call']
        if function_call["name"] == "call_google_places_api":
            place_type = json.loads(function_call["arguments"])["place_type"]
            places = call_google_places_api(user_id, place_type, food_preference)
            if places:  # If the list of places is not empty
                return f"Here are some places you might be interested in: {' '.join(places)}"
            else:
                return "I couldn't find any places of interest nearby."

    return "I am sorry, but I could not understand your request."



In [ ]:
from typing import Any, Union
import json

def provide_user_specific_recommendations(user_input: str, user_id: str) -> str:
    """
    ユーザーの入力に基づいて特定の推薦を提供します。

    Args:
        user_input (str): ユーザーからの入力。
        user_id (str): ユーザーID。

    Returns:
        str: レスポンスメッセージ。
    """
    customer_profile = fetch_customer_profile(user_id)
    if customer_profile is None:
        return "プロファイルが見つかりませんでした。ユーザーIDを確認してください。"

    customer_profile_str = json.dumps(customer_profile)

    food_preference = customer_profile.get('preferences', {}).get('food', [])[0] if customer_profile.get('preferences', {}).get('food') else None

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": f"あなたは洗練されたAIアシスタントであり、ユーザーの意図を検出し解釈する専門家です。タスクは、ユーザーが直接的または間接的な方法で表現したニーズを把握し、応答することです。微妙な手がかりを認識するのが得意です。例えば、ユーザーが「お腹が空いた」と発言した場合、レストランやカフェなどの近くの食事オプションを求めていると仮定するべきです。彼らが「疲れた」、「疲れた」と感じるか、長い旅を挙げた場合、これはホテルやゲストハウスなどの宿泊オプションのリクエストとして解釈します。しかし、ユーザーの意図が不明確であるか、複数の方法で解釈できる場合は、追加の説明を丁寧に求めることをためらってはいけません。{customer_profile_str}に記載されているユーザーの嗜好と過去の経験に基づいて、応答を調整してください。"
            },
            {"role": "user", "content": user_input}
        ],
        temperature=0,
        functions=[
            {
                "name": "call_google_places_api",
                "description": "この関数はGoogle Places APIを呼び出して、特定の場所のタイプを特定の場所の近くで見つけます。ユーザーがニーズ（例えば、お腹が空いている、疲れているなど）を表現した場合や特定のタイプの場所（例えば、レストランやホテル）を見つけたい場合に使用できます。",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "place_type": {
                            "type": "string",
                            "description": "検索する場所のタイプ。"
                        }
                    }
                },
                "result": {
                    "type": "array",
                    "items": {
                        "type": "string"
                    }
                }
            }
        ],
    )

    if 'message' in response.choices[0] and 'function_call' in response.choices[0]['message']:
        function_call = response.choices[0]['message']['function_call']
        if function_call["name"] == "call_google_places_api":
            place_type = json.loads(function_call["arguments"])["place_type"]
            places = call_google_places_api(user_id, place_type, food_preference)
            if places:
                return f"興味があるかもしれない場所は次のとおりです：{' '.join(places)}"
            else:
                return "近くで興味を持っているかもしれない場所は見つかりませんでした。"

    return "申し訳ありませんが、要求を理解できませんでした。"


# ユーザー固有の推薦を実行する

実行時、この関数はユーザーのプロフィールを取得し、AIモデルと対話し、モデルの応答を処理し、必要に応じてGoogle Places APIを呼び出し、最終的にユーザーの好みと場所に合わせた推薦のリストを返します。印刷される出力は、これらの個別の推薦から構成されます。

In [15]:
user_id = "user1234"  
user_input = "I'm hungry"  
output = provide_user_specific_recommendations(user_input, user_id)
print(output)

Here are some places you might be interested in: Mona Lisa Restaurant is a restaurant located at 353 Columbus Avenue #3907. It has a rating of 4.3 based on 1784 user reviews. Tommaso's Ristorante Italiano is a restaurant located at 1042 Kearny Street. It has a rating of 4.5 based on 732 user reviews.
